In [13]:
import pandas as pd

df_backstreetboys = pd.read_csv('Dados\Backstreet_Boys_Lyrics_score.csv') 
df_backstreetboys = df_backstreetboys.drop(columns=["Unnamed: 0", "Unnamed: 0.1"]) # Remove colunas desnecessárias

df_arcticmonkeys = pd.read_csv('Dados\lyrics.csv')

In [14]:
# Divide o dataset em 2, um para cada banda
df_bb_treino = df_backstreetboys.iloc[:49] # Seleciona as primeiras linhas do dataset
df_bb_teste = df_backstreetboys.iloc[50:100] # Seleciona as últimas linhas do dataset

df_am_treino = df_arcticmonkeys.iloc[:49] # Seleciona as primeiras linhas do dataset
df_am_teste = df_arcticmonkeys.iloc[50:100] # Seleciona as últimas linhas do dataset

In [15]:
# Probabilidade de ser de cada banda

p_am = len(df_am_treino) / (len(df_am_treino) + len(df_bb_treino))
p_bb = len(df_bb_treino) / (len(df_am_treino) + len(df_bb_treino))

In [16]:
def separador_palavra(text):
    return text.split() # Separa o texto em palavras a partir dos espaços

# Probabilidade de cada palavra aparecer em Arctic Monkeys
dict_am = {}
dict_palavras_am = {}

for musica_am in df_am_treino['lyrics']:
    musica_am = musica_am.lower().replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').replace(';', '').replace(':', '').replace('"', '').replace("'", '').replace('-', '').replace('[', '').replace(']', '').replace('\r\n', ' ')
    dict_am['palavras'] = separador_palavra(musica_am) # Aplica a função separador_palavra a cada linha da coluna lyrics
    df_am_novo = pd.DataFrame(dict_am)
    palavras_am = df_am_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_am = palavras_am.unique() # Remove palavras repetidas

    for palavra_am in palavras_unicas_am:
        if palavra_am not in dict_palavras_am:
            dict_palavras_am[palavra_am] = 1
        else:
            dict_palavras_am[palavra_am] += 1

dict_prob_am = {}

for palavra_am in dict_palavras_am:
    dict_prob_am[palavra_am] = dict_palavras_am[palavra_am] / len(df_am_treino)

In [17]:
# Probabilidade de cada palavra aparecer em Backstreets Boys
dict_bb = {}
dict_palavras_bb = {}

for musica_bb in df_bb_treino['Lyrics']:
    musica_bb = musica_bb.lower().replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').replace(';', '').replace(':', '').replace('"', '').replace("'", '').replace('-', '').replace('[', '').replace(']', '').replace('\r\n', ' ')
    dict_bb['palavras'] = separador_palavra(musica_bb) # Aplica a função separador_palavra a cada linha da coluna lyrics
    df_bb_novo = pd.DataFrame(dict_bb)
    palavras_bb = df_bb_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_bb = palavras_bb.unique() # Remove palavras repetidas

    for palavra_bb in palavras_unicas_bb:
        if palavra_bb not in dict_palavras_bb:
            dict_palavras_bb[palavra_bb] = 1
        else:
            dict_palavras_bb[palavra_bb] += 1

dict_prob_bb = {}

for palavra_bb in dict_palavras_bb:
    dict_prob_bb[palavra_bb] = dict_palavras_bb[palavra_bb] / len(df_bb_treino)

In [18]:
# Probabilidade de cada palavra aparecer em ambas as bandas
dict_palavras_both = {}

for palavra_am_new in dict_palavras_am:
    dict_palavras_both[palavra_am_new] = dict_palavras_am[palavra_am_new]

for palavra_bb_new in dict_palavras_bb:
    if palavra_bb_new not in dict_palavras_both:
        dict_palavras_both[palavra_bb_new] = dict_palavras_bb[palavra_bb_new]
    else:
        dict_palavras_both[palavra_bb_new] += dict_palavras_bb[palavra_bb_new]

dict_prob_both = {}

for palavra_both in dict_palavras_both:
    dict_prob_both[palavra_both] = dict_palavras_both[palavra_both] / (len(df_am_treino) + len(df_bb_treino))

In [21]:
# Probabilidade de ser do Arctic Monkeys dado que a palavra aparece
# Teorema de Bayes 
# P(A|B) = P(B|A) * P(A) / P(B)
dict_prob_am_dado_palavra = {}

for palavra in dict_prob_am:
    dict_prob_am_dado_palavra[palavra] = (dict_prob_am[palavra] * p_am) / dict_prob_both[palavra]

In [29]:
def classificador(dict_am, dict_bb, dict_both):
    certo = 0
    errado = 0

    for palavra in dict_both:
        if palavra not in dict_am or palavra not in dict_bb:
            continue
        if dict_am[palavra] > dict_bb[palavra]:
            if dict_prob_am_dado_palavra[palavra] > 0.5:
                certo += 1
            else:
                errado += 1
        else:
            if dict_prob_am_dado_palavra[palavra] < 0.5:
                certo += 1
            else:
                errado += 1
    
    return certo, errado

certo, errado = classificador(dict_prob_am, dict_prob_bb, dict_prob_both)

prob_certo = certo / (certo + errado)

prob_certo

0.7860869565217391